## Imports

In [37]:
import os
import numpy as np
import pandas as pd

from tqdm import tqdm
from dynamic_helper import *

import dynamic_helper_strats_pack as strats

In [38]:
pd.set_option("display.max_rows", 100)
pd.set_option("display.max_columns", None)
pd.set_option("display.width", None)
pd.set_option("display.max_colwidth", None)
pd.set_option("display.expand_frame_repr", False)
pd.set_option("display.multi_sparse", False)
pd.set_option("display.max_seq_items", None)
tqdm.pandas()

## Params

In [39]:
# FILLER
EQUITY_TYPE = 'despac'

## Load Data And Merge

In [40]:
stock_data = pd.read_parquet(f'../data/stock_data/{EQUITY_TYPE}/clean_stock_data.parquet')

if os.path.exists(f"../data/options_stock/{EQUITY_TYPE}/data.parquet"):
    data = pd.read_parquet(f"../data/options_stock/{EQUITY_TYPE}/data.parquet", engine="pyarrow")
    pd.concat([data.head(1), data.tail(1)])
else:
    # Merge data
    options_data = pd.read_parquet(f"../data/option_data/{EQUITY_TYPE}/clean_options_data.parquet")

    options_data = options_data.loc[options_data['volume'] >= 10]

    data = options_data.join(
        stock_data, how='left', on=['date', 'ticker']
    )

    # Add in distance from strike price
    data = data.reset_index()

    data['type_multiplier'] = np.where(data['option_type'] == 'C', 1, -1)
    data['intrisinc_value'] = (data['stock_price'] - data["strike"]) * data['type_multiplier']
    data['extrinsic_value_last'] = data['last'] - data['intrisinc_value']
    data['extrinsic_value_mark'] = data['mark'] - data['intrisinc_value']
    data['pct_from_strike'] = data['intrisinc_value'] / data["strike"]

    grouped_option_chains = data.groupby(by='id')
    data['daily_last_pct_change'] = grouped_option_chains['last'].pct_change().fillna(0)
    data['daily_mark_pct_change'] = grouped_option_chains['mark'].pct_change().fillna(0)

    data = data.set_index(['id', 'date'])

    # Convert to parquet
    data.to_parquet(f"../data/options_stock/{EQUITY_TYPE}/data.parquet", engine="pyarrow")

pd.concat([data.head(1), data.tail(1)])

,,ticker,expiration,strike,option_type,last,mark,bid,bid_size,ask,ask_size,volume,open_interest,implied_volatility,delta,gamma,theta,vega,rho,days_till_expiration,stock_price,type_multiplier,intrisinc_value,extrinsic_value_last,extrinsic_value_mark,pct_from_strike,daily_last_pct_change,daily_mark_pct_change
id,date,,,,,,,,,,,,,,,,,,,,,,,,,,,
SPCE191115C00005000,2019-10-28,SPCE,2019-11-15,5.0,C,7.20,6.85,4.5,6,9.2,1,300,0,2.18068,0.97807,0.00917,-0.00854,0.00137,0.00231,18,11.79,1,6.79,0.41,0.06,1.358,0.0,0.0
ZURA260618P00002500,2025-12-29,ZURA,2026-06-18,2.5,P,0.25,0.01,0.0,0,0.5,7,20,0,0.55145,-0.01224,0.01573,-0.00018,0.00116,-0.00035,171,5.35,-1,-2.85,3.10,2.86,-1.140,0.0,0.0


In [41]:
trade_filter = find_trades(stock_data, 3, 100, 'U')

pd.concat([trade_filter.head(1), trade_filter.tail(1)])

,ticker,episode_start_date,episode_end_date,start_price,end_price,days_to_threshold,move_pct
0,AEVA,2024-03-14,2024-03-19,0.9746,5.20,3,433.552227
230,ZURA,2025-09-25,2025-09-30,2.1000,4.33,3,106.190476


In [ ]:
# positions = strats.find_backspread_ids(trade_filter, data, 0, 25, max_strike_width_pct=0.03, price_col="mark")

# positions = strats.find_call_split_strangle_ids(
#     trade_filter=trade_filter,
#     options_data=data,
#     pct_target=0,                 # ATM call
#     days_till_expiration_target=30,
#     price_col="mark",
# ).sort_index()

positions = strats.find_call_split_strangle_ids(
    trade_filter=trade_filter,
    options_data=data,
    pct_target=0,                 # ATM call
    days_till_expiration_target=30,
    price_col="mark",
).sort_index()


positions = positions.rename_axis(index={"date": "start_date"})

pd.concat([positions.head(1), positions.tail(1)])

,,,ticker,quantity,direction
id,start_date,trade_filter_id,,,
ARBE250221C00005000,2025-01-06,6,ARBE,-1,S
ZURA251017P00002500,2025-09-30,230,ZURA,1,L


In [43]:
positon_keys = [t[:2] for t in list(positions.index)]
options_chain_raw = get_option_chains(positon_keys, data)

options_chain_raw.head(2)

,,,ticker,expiration,strike,option_type,last,mark,bid,bid_size,ask,ask_size,volume,open_interest,implied_volatility,delta,gamma,theta,vega,rho,days_till_expiration,stock_price,type_multiplier,intrisinc_value,extrinsic_value_last,extrinsic_value_mark,pct_from_strike,daily_last_pct_change,daily_mark_pct_change,entry_last_price,entry_mark_price,trade_last_pct_change,trade_mark_pct_change
id,start_date,date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
ARBE250221C00005000,2025-01-06,2025-01-06,ARBE,2025-02-21,5.0,C,1.11,1.07,1.00,113,1.15,245,4177,4666,2.45384,0.57365,0.11253,-0.01500,0.00557,0.00155,46,4.00,1,-1.00,2.11,2.07,-0.200,4.842105,4.35000,1.11,1.07,0.000000,0.00000
ARBE250221C00005000,2025-01-06,2025-01-07,ARBE,2025-02-21,5.0,C,0.90,0.88,0.75,726,1.00,25,4222,5539,2.37580,0.53261,0.12676,-0.01399,0.00525,0.00139,45,3.76,1,-1.24,2.14,2.12,-0.248,-0.189189,-0.17757,1.11,1.07,-0.189189,-0.17757


In [44]:
positions_for_join = positions.copy()
positions_for_join["trade_filter_id"] = positions_for_join.index.get_level_values("trade_filter_id")
positions_for_join = positions_for_join.droplevel("trade_filter_id").drop(columns=["ticker"])


informative_option_chains = options_chain_raw.join(
    positions_for_join,
    on=["id", "start_date"],
    how="left"
)

informative_option_chains = informative_option_chains.reset_index().set_index(["trade_filter_id", "id", "date"]).sort_index()

option_chains = informative_option_chains[['stock_price', 'strike', 'option_type', 'quantity', 'direction', 'last', 'mark', 'entry_last_price', 'entry_mark_price']]

option_chains.head(2)

,,,stock_price,strike,option_type,quantity,direction,last,mark,entry_last_price,entry_mark_price
trade_filter_id,id,date,,,,,,,,,
6,ARBE250221C00005000,2025-01-06,4.00,5.0,C,-1,S,1.11,1.07,1.11,1.07
6,ARBE250221C00005000,2025-01-07,3.76,5.0,C,-1,S,0.90,0.88,1.11,1.07


In [45]:
legs = pnl_transaction(option_chains, price_col="mark")

legs.head(2)


,,,stock_price,strike,option_type,quantity,direction,last,mark,entry_last_price,entry_mark_price,entry_price,price,side_sign,quantity_abs,multiplier_used,qty_direction_mismatch,leg_cost,leg_value,pnl,entry_notional_abs,pnl_pct
trade_filter_id,id,date,,,,,,,,,,,,,,,,,,,,
6,ARBE250221C00005000,2025-01-06,4.00,5.0,C,-1,S,1.11,1.07,1.11,1.07,1.07,1.07,-1,1.0,100.0,False,-107.0,-107.0,0.0,107.0,0.00000
6,ARBE250221C00005000,2025-01-07,3.76,5.0,C,-1,S,0.90,0.88,1.11,1.07,1.07,0.88,-1,1.0,100.0,False,-107.0,-88.0,19.0,107.0,0.17757


In [46]:
trade_ts_stopped = pnl_trade(
    option_chains,
    price_col="mark",
    take_profit=4,
    truncate=True,
)

trade_ts_stopped.head(2)

,,trade_pnl,trade_date,sell_date,holding_days,trade_cost,trade_cost_abs,trade_entry_notional_abs,trade_pnl_pct,trade_value,trade_pnl_pct_cost,trade_n_legs,trade_n_contracts,trade_n_tickers,multi_ticker_trade,max_trade_loss,max_trade_gain,unbounded_risk,unbounded_gain,gain_loss_ratio,trade_pnl_over_cost_abs,trade_cost_pct_of_gross
trade_filter_id,date,,,,,,,,,,,,,,,,,,,,,
6,2025-01-06,0.0,2025-01-06,2025-02-21,46,-2.0,212.0,212.0,0.000000,-2.0,0.0,3,3.0,1,False,248.0,252.0,False,False,0.000000,0.000000,-0.009434
6,2025-01-07,16.0,2025-01-06,2025-02-21,46,-2.0,212.0,212.0,0.075472,14.0,8.0,3,3.0,1,False,248.0,252.0,False,False,0.064516,0.075472,-0.009434


In [47]:
lasts = trade_ts_stopped.groupby(level='trade_filter_id').last()

lasts.tail(2)

,trade_pnl,trade_date,sell_date,holding_days,trade_cost,trade_cost_abs,trade_entry_notional_abs,trade_pnl_pct,trade_value,trade_pnl_pct_cost,trade_n_legs,trade_n_contracts,trade_n_tickers,multi_ticker_trade,max_trade_loss,max_trade_gain,unbounded_risk,unbounded_gain,gain_loss_ratio,trade_pnl_over_cost_abs,trade_cost_pct_of_gross
trade_filter_id,,,,,,,,,,,,,,,,,,,,,
222,-7.0,2022-02-03,2022-03-10,35,-22.0,48.0,48.0,-0.145833,-29.0,-0.318182,3,3.0,1,False,228.0,272.0,False,False,0.0,-0.145833,-0.458333
230,-19.0,2025-09-30,2025-10-16,16,-32.0,74.0,74.0,-0.256757,-51.0,-0.593750,3,3.0,1,False,218.0,282.0,False,False,0.0,-0.256757,-0.432432


In [48]:
def build_100_portfolio(
    trade_ts: pd.DataFrame,
    capital_per_trade: float = 100.0,
    basis: str = "risk",   # "risk" | "gross_premium" | "net_cost"
    drop_unbounded: bool = True,
    smooth_window: int = 21,
):
    """
    Build portfolio series assuming capital_per_trade per trade using a chosen return basis.

    basis:
      - "risk":          return = trade_pnl / max_trade_loss            (best for intuition about max loss)
      - "gross_premium": return = trade_pnl / trade_cost_abs            (your previous trade_pnl_pct style)
      - "net_cost":      return = trade_pnl / abs(trade_cost)           (can blow up if net cost ~0)

    drop_unbounded:
      - if True and basis="risk", drops trades where max_trade_loss is inf/NaN/<=0

    Outputs:
      ts:         trade-level daily with return_basis + pnl_100
      trade_meta: per trade with final pnl_100 + final return
      port:       daily portfolio series + smoothed unrealized + rolling stats
    """
    if not isinstance(trade_ts.index, pd.MultiIndex):
        raise ValueError("Expected trade_ts indexed by ['trade_filter_id', 'date']")

    req = {"trade_pnl", "trade_date", "sell_date"}
    missing = req.difference(trade_ts.columns)
    if missing:
        raise ValueError(f"trade_ts missing required columns: {sorted(missing)}")

    ts = trade_ts.reset_index().copy()
    ts["date"] = pd.to_datetime(ts["date"])
    ts["trade_date"] = pd.to_datetime(ts["trade_date"])
    ts["sell_date"] = pd.to_datetime(ts["sell_date"])

    pnl = pd.to_numeric(ts["trade_pnl"], errors="coerce")

    # Choose denominator
    if basis == "risk":
        if "max_trade_loss" not in ts.columns:
            raise ValueError("basis='risk' requires 'max_trade_loss' in trade_ts output")
        denom = pd.to_numeric(ts["max_trade_loss"], errors="coerce")

        if drop_unbounded:
            ok = np.isfinite(denom) & (denom > 0)
            keep_tids = ts.loc[ok, "trade_filter_id"].unique()
            ts = ts[ts["trade_filter_id"].isin(keep_tids)].copy()
            pnl = pd.to_numeric(ts["trade_pnl"], errors="coerce")
            denom = pd.to_numeric(ts["max_trade_loss"], errors="coerce")

    elif basis == "gross_premium":
        if "trade_cost_abs" in ts.columns:
            denom = pd.to_numeric(ts["trade_cost_abs"], errors="coerce")
        elif "trade_entry_notional_abs" in ts.columns:
            denom = pd.to_numeric(ts["trade_entry_notional_abs"], errors="coerce")
        else:
            raise ValueError("basis='gross_premium' requires 'trade_cost_abs' or 'trade_entry_notional_abs'")

    elif basis == "net_cost":
        if "trade_cost" not in ts.columns:
            raise ValueError("basis='net_cost' requires 'trade_cost'")
        denom = pd.to_numeric(ts["trade_cost"], errors="coerce").abs()

    else:
        raise ValueError("basis must be one of: 'risk', 'gross_premium', 'net_cost'")

    denom = denom.replace(0, np.nan)

    ts["return_basis"] = pnl / denom
    ts["pnl_100"] = ts["return_basis"] * float(capital_per_trade)

    ts = ts.sort_values(["trade_filter_id", "date"], kind="mergesort")

    # Per-trade snapshot
    first = ts.groupby("trade_filter_id", sort=False).first()
    last = ts.groupby("trade_filter_id", sort=False).last()

    trade_meta = pd.DataFrame(index=first.index)
    trade_meta.index.name = "trade_filter_id"
    trade_meta["trade_date"] = first["trade_date"]
    trade_meta["sell_date"] = first["sell_date"]
    trade_meta["final_pnl_100"] = last["pnl_100"]
    trade_meta["final_return"] = trade_meta["final_pnl_100"] / float(capital_per_trade)

    # Realized: book on sell_date
    realized_daily = trade_meta.groupby("sell_date", sort=False)["final_pnl_100"].sum().rename("realized_daily")

    # Unrealized: sum pnl_100 for open trades
    open_mask = ts["date"] < ts["sell_date"]
    unrealized = ts.loc[open_mask].groupby("date", sort=False)["pnl_100"].sum().rename("unrealized")

    open_trades = ts.loc[open_mask].groupby("date", sort=False)["trade_filter_id"].nunique().rename("open_trades")

    # Capital entered: $100 per trade at trade_date
    entered_daily = (
        trade_meta.groupby("trade_date", sort=False)
        .size()
        .astype("float64")
        .mul(float(capital_per_trade))
        .rename("capital_entered_daily")
    )

    all_dates = pd.Index(sorted(ts["date"].unique()), name="date")
    port = pd.DataFrame(index=all_dates)

    port["realized_daily"] = realized_daily.reindex(all_dates).fillna(0.0)
    port["realized_cum"] = port["realized_daily"].cumsum()

    port["unrealized"] = unrealized.reindex(all_dates).fillna(0.0)
    port["unrealized_smooth"] = port["unrealized"].rolling(int(smooth_window), min_periods=1).mean()

    port["total_equity"] = port["realized_cum"] + port["unrealized"]

    peak = port["total_equity"].cummax()
    port["drawdown"] = port["total_equity"] - peak

    port["open_trades"] = open_trades.reindex(all_dates).fillna(0).astype("int64")
    port["open_capital"] = port["open_trades"] * float(capital_per_trade)

    port["capital_entered_daily"] = entered_daily.reindex(all_dates).fillna(0.0)
    port["cum_capital_entered"] = port["capital_entered_daily"].cumsum()

    # Time-weighted deployment: "capital-days"
    # (sum of open_capital each day; gives intuition for how much you had deployed over time)
    port["capital_days"] = port["open_capital"].cumsum()

    # Rolling realized expectancy & win rate (on closed trades stream)
    closed = trade_meta.sort_values("sell_date").copy()
    closed["win"] = (closed["final_pnl_100"] > 0).astype("int64")
    closed["loss"] = (closed["final_pnl_100"] < 0).astype("int64")

    # Rolling over last N closed trades (not calendar time)
    N = 100
    closed["roll_avg_pnl_100"] = closed["final_pnl_100"].rolling(N, min_periods=10).mean()
    closed["roll_win_rate"] = closed["win"].rolling(N, min_periods=10).mean()

    return ts, trade_meta, port, closed

import matplotlib.pyplot as plt

import matplotlib.pyplot as plt

def dashboard_strategy_health(
    trade_ts: pd.DataFrame,
    capital_per_trade: float = 100.0,
    default_basis: str = "risk",
):
    """
    Interactive dashboard with:
      - Cumulative realized
      - Unrealized (raw + smoothed)
      - Total equity + drawdown
      - Monthly realized bars
      - Distribution of closed trade returns
      - Distribution of open-trade returns at selected date
      - Rolling expectancy + win rate (last 100 closed trades)
      - Diagnostic tables: biggest open losers/winners

    Fix: uses observe() instead of nested interactive_output() so it always renders.
    """
    try:
        import ipywidgets as widgets
        from IPython.display import display, clear_output
    except Exception as e:
        raise RuntimeError("ipywidgets required (pip install ipywidgets) and restart kernel.") from e

    basis_dd = widgets.Dropdown(
        options=[
            ("Return on Max Loss (recommended)", "risk"),
            ("Return on Gross Premium", "gross_premium"),
            ("Return on Net Cost (dangerous near 0)", "net_cost"),
        ],
        value=default_basis,
        description="Basis",
        layout=widgets.Layout(width="60%"),
    )

    smooth = widgets.IntSlider(
        value=21, min=1, max=126, step=5,
        description="Smooth",
        continuous_update=False,
        layout=widgets.Layout(width="60%"),
    )

    controls = widgets.VBox([basis_dd, smooth])
    out = widgets.Output()

    def _render(_=None):
        with out:
            clear_output(wait=True)

            ts, trade_meta, port, closed = build_100_portfolio(
                trade_ts,
                capital_per_trade=capital_per_trade,
                basis=basis_dd.value,
                drop_unbounded=True if basis_dd.value == "risk" else False,
                smooth_window=int(smooth.value),
            )

            dates = list(port.index)
            if len(dates) == 0:
                display(pd.DataFrame({"error": ["No dates found in portfolio series (port is empty)."]}))
                return

            date_slider = widgets.SelectionSlider(
                options=[(d.strftime("%Y-%m-%d"), d) for d in dates],
                value=dates[-1],
                description="As of",
                continuous_update=False,
                layout=widgets.Layout(width="95%"),
            )

            topn = widgets.IntSlider(
                value=15, min=5, max=50, step=5,
                description="Top N",
                continuous_update=False,
                layout=widgets.Layout(width="60%"),
            )

            inner_out = widgets.Output()
            ui = widgets.VBox([date_slider, topn, inner_out])
            display(ui)

            def _draw(_=None):
                with inner_out:
                    clear_output(wait=True)

                    as_of = pd.to_datetime(date_slider.value)
                    top_n = int(topn.value)

                    port_asof = port.loc[:as_of].copy()

                    fig, axes = plt.subplots(3, 2, figsize=(14, 12))

                    ax = axes[0, 0]
                    ax.plot(port_asof.index, port_asof["realized_cum"])
                    ax.set_title(f"Cumulative Realized PnL (${capital_per_trade:.0f} per trade, basis={basis_dd.value})")
                    ax.set_xlabel("Date"); ax.set_ylabel("$")

                    ax = axes[0, 1]
                    ax.plot(port_asof.index, port_asof["unrealized"], label="Raw Unrealized")
                    ax.plot(port_asof.index, port_asof["unrealized_smooth"], label=f"{int(smooth.value)}d Mean")
                    ax.legend()
                    ax.set_title("Unrealized PnL (raw + smoothed)")
                    ax.set_xlabel("Date"); ax.set_ylabel("$")

                    ax = axes[1, 0]
                    ax.plot(port_asof.index, port_asof["total_equity"], label="Total Equity")
                    ax.plot(port_asof.index, port_asof["drawdown"], label="Drawdown")
                    ax.legend()
                    ax.set_title("Total Equity + Drawdown")
                    ax.set_xlabel("Date"); ax.set_ylabel("$")

                    ax = axes[1, 1]
                    ax.plot(port_asof.index, port_asof["open_trades"])
                    ax.set_title("Open Trades (count)")
                    ax.set_xlabel("Date"); ax.set_ylabel("# trades")

                    ax = axes[2, 0]
                    monthly = port_asof["realized_daily"].resample("ME").sum()
                    ax.bar(monthly.index, monthly.values, width=20)
                    ax.set_title("Monthly Realized PnL")
                    ax.set_xlabel("Month"); ax.set_ylabel("$")

                    ax = axes[2, 1]
                    ax.plot(port_asof.index, port_asof["cum_capital_entered"], label="Cumulative Capital Entered")
                    ax.plot(port_asof.index, port_asof["capital_days"], label="Capital-Days (time-weighted)")
                    ax.legend()
                    ax.set_title("Deployment (how much you put to work)")
                    ax.set_xlabel("Date"); ax.set_ylabel("$ (units)")

                    plt.tight_layout()
                    plt.show()

                    # --- Capital Efficiency Curve: Cum $ Entered (x) vs Cum Realized PnL (y) ---
                    if "max_trade_loss" in ts.columns:
                        # max loss is constant per trade, so take the first value per trade_filter_id
                        max_loss_by_tid = (
                            ts.groupby("trade_filter_id", sort=False)["max_trade_loss"]
                            .first()
                            .replace([np.inf, -np.inf], np.nan)
                        )

                        # Sum max loss on the day each trade is entered
                        tmp = trade_meta.copy()
                        tmp["max_trade_loss"] = tmp.index.map(max_loss_by_tid)

                        max_loss_entered_daily = (
                            tmp.dropna(subset=["max_trade_loss"])
                            .groupby("trade_date", sort=False)["max_trade_loss"]
                            .sum()
                            .rename("max_loss_entered_daily")
                        )

                        # Align to portfolio dates and cum-sum
                        cum_max_loss_entered = (
                            max_loss_entered_daily.reindex(port_asof.index).fillna(0.0).cumsum()
                        )

                        x = cum_max_loss_entered.to_numpy(dtype="float64")
                        y = port_asof["realized_cum"].to_numpy(dtype="float64")

                        # Guard against weird empties
                        if len(x) > 0 and len(y) > 0:
                            plt.figure(figsize=(14, 4))
                            plt.plot(x, y)
                            plt.axhline(0)
                            plt.axvline(0)

                            last_x = float(x[-1])
                            last_y = float(y[-1])
                            plt.scatter([last_x], [last_y])

                            roi = (last_y / last_x) if last_x > 0 else np.nan

                            plt.title("Cumulative Realized PnL vs Cumulative Max Loss Entered")
                            plt.xlabel("Cumulative Max Loss Entered ($)")
                            plt.ylabel("Cumulative Realized PnL ($)")

                            if np.isfinite(roi):
                                plt.text(last_x, last_y, f"  ROI on max-loss entered: {roi:.2%}")

                            plt.tight_layout()
                            plt.show()
                    else:
                        print("Note: 'max_trade_loss' not present in ts, skipping Cum Max Loss Entered plot.")

                    # --- Risk-Days Efficiency Curve: Cum(Open Max Loss) (x) vs Cum Realized PnL (y) ---


                    if "max_trade_loss" in ts.columns:
                        # Per-trade constant max loss (take first row per trade_filter_id)
                        max_loss_by_tid = (
                            ts.groupby("trade_filter_id", sort=False)["max_trade_loss"]
                            .first()
                            .replace([np.inf, -np.inf], np.nan)
                        )
                        max_loss_by_tid = pd.to_numeric(max_loss_by_tid, errors="coerce")

                        # Use your ts rows (one per trade per date) to identify which trades are open each day
                        open_rows = ts[(ts["date"] <= as_of) & (ts["date"] < ts["sell_date"])].copy()
                        open_rows["max_trade_loss"] = open_rows["trade_filter_id"].map(max_loss_by_tid)

                        # Daily sum of max loss across OPEN trades
                        open_max_loss_daily = (
                            open_rows.groupby("date", sort=False)["max_trade_loss"]
                            .sum()
                            .rename("open_max_loss")
                        )

                        # Align to portfolio dates and cum-sum to get risk-days
                        open_max_loss_daily = open_max_loss_daily.reindex(port_asof.index).fillna(0.0)
                        risk_days = open_max_loss_daily.cumsum()

                        # Plot: realized cum PnL vs cumulative risk-days
                        x = risk_days.to_numpy(dtype="float64")
                        y = port_asof["realized_cum"].to_numpy(dtype="float64")

                        if len(x) > 0 and len(y) > 0:
                            plt.figure(figsize=(14, 4))
                            plt.plot(x, y)
                            plt.axhline(0)
                            plt.axvline(0)

                            last_x = float(x[-1])
                            last_y = float(y[-1])
                            plt.scatter([last_x], [last_y])

                            # Note: units are $ / ($-day) = 1/day (not a % ROI)
                            eff = (last_y / last_x) if last_x > 0 else np.nan

                            plt.title("Better Realized PnL vs Cumulative Open Max Loss (Risk-Days)")
                            plt.xlabel("Cumulative Open Max Loss ($-days)")
                            plt.ylabel("Cumulative Realized PnL ($)")

                            if np.isfinite(eff):
                                plt.text(last_x, last_y, f"  Profit per $-risk-day: {eff:.6g}")

                            plt.tight_layout()
                            plt.show()

                            # Optional: also show the open-risk time series itself (often very informative)
                            plt.figure(figsize=(14, 3))
                            plt.plot(port_asof.index, open_max_loss_daily)
                            plt.title("Open Max Loss ($) per Day (Defined-Risk Exposure)")
                            plt.xlabel("Date")
                            plt.ylabel("Open Max Loss ($)")
                            plt.tight_layout()
                            plt.show()
                    else:
                        print("Note: 'max_trade_loss' not present in ts; skipping open-risk risk-days plot.")

                    # --- Risk-Days Efficiency Curve: Cum(Open Max Loss) (x) vs Cum Realized PnL (y) ---
                    if "max_trade_loss" in ts.columns:
                        # Per-trade constant max loss (take first row per trade_filter_id)
                        max_loss_by_tid = (
                            ts.groupby("trade_filter_id", sort=False)["max_trade_loss"]
                            .first()
                            .replace([np.inf, -np.inf], np.nan)
                        )
                        max_loss_by_tid = pd.to_numeric(max_loss_by_tid, errors="coerce")

                        # Use your ts rows (one per trade per date) to identify which trades are open each day
                        open_rows = ts[(ts["date"] <= as_of) & (ts["date"] < ts["sell_date"])].copy()
                        open_rows["max_trade_loss"] = open_rows["trade_filter_id"].map(max_loss_by_tid)

                        # Daily sum of max loss across OPEN trades
                        open_max_loss_daily = (
                            open_rows.groupby("date", sort=False)["max_trade_loss"]
                            .sum()
                            .rename("open_max_loss")
                        )

                        # Align to portfolio dates and cum-sum to get risk-days
                        open_max_loss_daily = open_max_loss_daily.reindex(port_asof.index).fillna(0.0)
                        risk_days = open_max_loss_daily.cumsum()

                        # Plot: realized cum PnL vs cumulative risk-days
                        x = risk_days.to_numpy(dtype="float64")
                        y = port_asof["realized_cum"].to_numpy(dtype="float64")

                        if len(x) > 0 and len(y) > 0:
                            plt.figure(figsize=(14, 4))
                            plt.plot(x, y)
                            plt.axhline(0)
                            plt.axvline(0)

                            last_x = float(x[-1])
                            last_y = float(y[-1])
                            plt.scatter([last_x], [last_y])

                            # Note: units are $ / ($-day) = 1/day (not a % ROI)
                            eff = (last_y / last_x) if last_x > 0 else np.nan

                            plt.title("Cumulative Realized PnL vs Cumulative Open Max Loss (Risk-Days)")
                            plt.xlabel("Cumulative Open Max Loss ($-days)")
                            plt.ylabel("Cumulative Realized PnL ($)")

                            if np.isfinite(eff):
                                plt.text(last_x, last_y, f"  Profit per $-risk-day: {eff:.6g}")

                            plt.tight_layout()
                            plt.show()

                            # Optional: also show the open-risk time series itself (often very informative)
                            plt.figure(figsize=(14, 3))
                            plt.plot(port_asof.index, open_max_loss_daily)
                            plt.title("Open Max Loss ($) per Day (Defined-Risk Exposure)")
                            plt.xlabel("Date")
                            plt.ylabel("Open Max Loss ($)")
                            plt.tight_layout()
                            plt.show()
                    else:
                        print("Note: 'max_trade_loss' not present in ts; skipping open-risk risk-days plot.")



                    x = port_asof["cum_capital_entered"].to_numpy(dtype="float64")
                    y = port_asof["realized_cum"].to_numpy(dtype="float64")

                    # Guard against empty slices (rare but safe)
                    if len(x) > 0 and len(y) > 0:
                        plt.figure(figsize=(14, 4))
                        plt.plot(x, y)
                        plt.axhline(0)

                        # Mark latest point + annotate ROI on entered capital
                        last_x = float(x[-1])
                        last_y = float(y[-1])
                        roi = (last_y / last_x) if last_x > 0 else np.nan

                        plt.scatter([last_x], [last_y])
                        plt.title("Cumulative Realized PnL vs Cumulative Capital Entered")
                        plt.xlabel("Cumulative Capital Entered ($)")
                        plt.ylabel("Cumulative Realized PnL ($)")

                        if np.isfinite(roi):
                            plt.text(last_x, last_y, f"  ROI on entered capital: {roi:.2%}")

                        plt.tight_layout()
                        plt.show()


                    # Rolling trade-stream metrics
                    closed_asof = closed[closed["sell_date"] <= as_of].copy()
                    if not closed_asof.empty:
                        plt.figure(figsize=(14, 3))
                        plt.plot(closed_asof["sell_date"], closed_asof["roll_avg_pnl_100"])
                        plt.title("Rolling Avg PnL per Closed Trade (last 100 trades)")
                        plt.xlabel("Sell Date"); plt.ylabel("$ per trade")
                        plt.show()

                        plt.figure(figsize=(14, 3))
                        plt.plot(closed_asof["sell_date"], closed_asof["roll_win_rate"])
                        plt.title("Rolling Win Rate (last 100 closed trades)")
                        plt.xlabel("Sell Date"); plt.ylabel("Win Rate")
                        plt.ylim(0, 1)
                        plt.show()

                        plt.figure(figsize=(14, 3))
                        x = closed_asof["final_return"].replace([np.inf, -np.inf], np.nan).dropna() * 100.0
                        plt.hist(x, bins=40)
                        plt.title("Distribution: Closed Trade Return (%)")
                        plt.xlabel("Return %"); plt.ylabel("Count")
                        plt.show()

                    # Open trade distribution + diagnostic tables at as_of
                    snap = ts[ts["date"] == as_of].copy()
                    if snap.empty:
                        display(pd.DataFrame({"note": ["No trade rows for this date."]}))
                        return

                    snap["is_open"] = snap["date"] < snap["sell_date"]
                    open_snap = snap[snap["is_open"]].copy()

                    if not open_snap.empty:
                        plt.figure(figsize=(14, 3))
                        x = open_snap["return_basis"].replace([np.inf, -np.inf], np.nan).dropna() * 100.0
                        plt.hist(x, bins=40)
                        plt.title("Distribution: Open Trades Return (%) at selected date")
                        plt.xlabel("Return %"); plt.ylabel("Count")
                        plt.show()

                        show = open_snap[["trade_filter_id", "trade_date", "sell_date", "pnl_100", "return_basis"]].copy()
                        show["return_%"] = show["return_basis"] * 100.0
                        show = show.drop(columns=["return_basis"])

                        print("Open trades: biggest losers")
                        display(show.sort_values("pnl_100").head(top_n))

                        print("Open trades: biggest winners")
                        display(show.sort_values("pnl_100", ascending=False).head(top_n))
                    else:
                        display(pd.DataFrame({"note": ["No open trades on this date."]}))

                    # Summary
                    row = port.loc[as_of]
                    summary = pd.DataFrame(
                        {
                            "as_of": [as_of],
                            "realized_cum_$": [row["realized_cum"]],
                            "unrealized_$": [row["unrealized"]],
                            "unrealized_smooth_$": [row["unrealized_smooth"]],
                            "total_equity_$": [row["total_equity"]],
                            "drawdown_$": [row["drawdown"]],
                            "open_trades": [row["open_trades"]],
                            "cum_capital_entered_$": [row["cum_capital_entered"]],
                        }
                    )
                    display(summary)

            # Wire updates + initial render
            date_slider.observe(_draw, names="value")
            topn.observe(_draw, names="value")
            _draw()

    # Wire top-level updates + initial render
    basis_dd.observe(_render, names="value")
    smooth.observe(_render, names="value")

    display(controls, out)
    _render()

dashboard_strategy_health(
    trade_ts_stopped,      # or trade_ts
    capital_per_trade=100.0,
    default_basis="risk",  # <-- start here; it matches your intuition much better
)



Output()